# Architecture explanation

Please play the video. I have explained every thing in the video including all the script and folder architecture

![title](re.svg)

# depth and size detcetion of the potholes

The main file of our work is that one. Is where we used the point cloud data of the road surface and projected it on the images to get the points inside each bounding box, which gave us the shape of the potholes. Using these techniques, we were able to estimate the depth and area of the potholes, and thus the magnitude of the potholes.

In [12]:
%cd yolov7/

[Errno 2] No such file or directory: 'yolov7/'
/home/anas/Desktop/HACTHON/yolov7


# imports

In [13]:

from plyfile import PlyData, PlyElement
import numpy as np
import open3d as o3
import cv2
import os
import re
import copy

# Read points cloud/ intrinsic /extrinsic parameter of the camera

For both scene1, and 2 we have created the 3D point cloud using colmap open source software. Don't worry, the software provide a python API to automatise the process of creting those clouds. Colmap use structure from motion (SFM) approach to reconstruct the points cloud from the provided images. 

The two point clooud for both scenes are in folders S1 and S2. Make sure that they are exist.

In [132]:
 
Scene1 = PlyData.read('s1/PointcS1')
Scene2 = PlyData.read('s2/PointcS2')
s1_x = Scene1.elements[0].data['x']
s1_y = Scene1.elements[0].data['y']
s1_z = Scene1.elements[0].data['z']

s2_x = Scene2.elements[0].data['x']
s2_y = Scene2.elements[0].data['y']
s2_z = Scene2.elements[0].data['z']

sc1 = np.array([s1_x,s1_y,s1_z]).T
sc2 = np.array([s2_x,s2_y,s2_z]).T

sc1_p = o3.geometry.PointCloud()
sc1_p.points = o3.utility.Vector3dVector(sc1)

sc2_p = o3.geometry.PointCloud()
sc2_p.points = o3.utility.Vector3dVector(sc2)

In [142]:
o3.visualization.draw_geometries([sc2_p])

Read extrinsic parameters (contain cameras position and its orientation)

In [135]:

with open('s2/images.txt','r') as line:
    f=line.read().splitlines()

Read intrinsic parameters (contain the focal of the camera and the principal points also)

In [136]:
with open('s2/cameras.txt','r') as line:
    d=line.read().splitlines()

Process intrinsic and extrinsic parameter. Convert them to numbers and pick only the position and orientation of the cameras + the focal + the principal points. 

In [17]:
# This cell extract the intinsic and extrinsic parameters.

images = sorted(os.listdir('Challenge/png/Scene2'))
path1 = 'anas1/exp/labels'
path = 'Challenge/png/Scene2'

def string_to_float_list(string):
    float_numbers = [float(x) for x in re.findall(r'[+-]?\d+\.\d+', string)]
    return float_numbers

#intrinsic cameras parameters
intrinsic_list =[]
for ele in d[3:]:
    intrinsic_list.append(string_to_float_list(ele[16:])[0])

# extract enrtinsic cameras parameteres
extrisic_list =[]

for image_name in images:
    for ele in f:
        if(image_name in ele):
            extrisic_list.append(string_to_float_list(ele[1:-9]))
            

We present here some function that will serve us in the process of finding the depth of the potholes. I want to mention here on important thing !!!!!!.

the results of calculating depth, size, bounding box, evnt the calculated points cloud are all in pixel unit. We need to convert it to (cm) to give senses to results output. One of the approach is to calculate the ratio which we need to know the physical size of the object being imaged and the number of pixels that represent that object in the image.

For example, if you know that an object in the real world is 10 cm long and it's represented by 100 pixels in the image, you can convert pixels to centimeters using the following formula:

ratio = real_world_size_in_cm / pixels_in_image

so we need to multiply the depth ans size and ... with the ratio. 

Here we have approximate manually one pothole (just by eye 27 cm) and we use its bounding box to find pixel number

In [137]:
real = 27
pixel = 130
ratio = 27/130

# Draw the bounding box on the image and show the depth and size after calculation.
def draw_bounding_box_with_size_and_depth(img, x_min, y_min, x_max, y_max, depth, proj):
    # Draw the bounding box on the image
    cv2.rectangle(img, (x_min, y_min), (x_max, y_max), (0, 255, 0), 2)
    # Calculate the size and depth of the bounding box
    width = x_max - x_min
    height = y_max - y_min
    size = width * height * ratio
    size = 'size:'+str(round(size,1))+'cm2'
    depth = 'depth:'+str(depth)+'cm'
    # Add text on the bounding box
    font = cv2.FONT_HERSHEY_SIMPLEX
    cv2.putText(img, size, (x_min + 5, y_max + 20), font, 1, (255,255,255), 2, cv2.LINE_AA)
    cv2.putText(img, str(depth), (x_min + 5, y_max + 40), font, 1, (255,255,255), 2, cv2.LINE_AA)
    
    for point in proj:
        x, y = point[0], point[1]
        cv2.circle(frame, (int(x), int(y)), 5, (0, 0, 255), -1)
    return img

# bounding box convert from x,y,width,high to two x,y and x1,y1 for cv2 drawing.
def xywh2xyxy(x):
    # Convert nx4 boxes from [x, y, w, h] to [x1, y1, x2, y2] where xy1=top-left, xy2=bottom-right
    y =  np.copy(x)
    y[:, 0] = x[:, 0] - x[:, 2] / 2  # top left x
    y[:, 1] = x[:, 1] - x[:, 3] / 2  # top left y
    y[:, 2] = x[:, 0] + x[:, 2] / 2  # bottom right x
    y[:, 3] = x[:, 1] + x[:, 3] / 2  # bottom right y
    return y

# This function calculate the inverse of the homogenous matrix
def invert_ht(ht):
    ht = np.tile(ht, [1, 1, 1])
    iht = np.tile(np.identity(4), [ht.shape[0], 1, 1])
    iht[..., :3, :3] = ht[..., :3, :3].transpose(0, 2, 1)
    iht[..., :3, [3]] = -np.matmul(iht[..., :3, :3], ht[..., :3, [3]])
    return iht.squeeze()


Aligning the points cloud with the cooresponding image based on the extrinsic cameras parameter.
Example aligning the points cloud with image '0001.png' is to use it is  extrinsic parameter
which contain its position and orientation by a homogunous transformation matrix which it help to transform 
any points cloud to the willing distance and to the willing direction just by simple multiplication.
I left you here a link to read more http://motion.cs.illinois.edu/RoboticSystems/CoordinateTransformations.html
The aim by this aliginement is to give sense to our projection because we can not project the points clouds
on the image without being both in same reference coordinate.

In [138]:

def Homogenous_transformation(extrinsic, cloud1):
    cloud = copy.deepcopy(cloud1)
    transformation = np.identity(4)
    # Transform the point cloud using the extrinsic parameters
    R = cloud.get_rotation_matrix_from_quaternion(extrinsic[:4])
    transformation[:3,:3] = R
    transformation[:3,3] = [extrinsic[4], extrinsic[5], extrinsic[6]]
    cloud = cloud.transform(transformation)
    cloud = np.array(cloud.points)
    
    return cloud

This a backward function which back from the image reference system to the body reference system. why using this. 
because to calculate the depth of the potholes we need to considere the whole reference system which make sence
comparing if we compute it just for the image reference system.

In [139]:

def inverse_Homogenous_transformation(extrinsic, cloud1):
    cloud2 = o3.geometry.PointCloud()
    cloud2.points = o3.utility.Vector3dVector(cloud1)
    cloud = copy.deepcopy(cloud2)
    transformation = np.identity(4)
    # Transform the point cloud using the extrinsic parameters
    R = cloud.get_rotation_matrix_from_quaternion(extrinsic[:4])
    transformation[:3,:3] = R
    transformation[:3,3] = [extrinsic[4], extrinsic[5], extrinsic[6]]
    inv_T = invert_ht(transformation)
    cloud = cloud.transform(inv_T)
    cloud = np.array(cloud.points)
    
    return cloud

After alignement, the projection of the points cloud on the image and also provide the 3D points included in each bounding box [IMPORTANT] 
because z is the depth of the so we can deduce the of the potoles by [IMPORTANT] calculating the min of z of all the points inside the bounding box [IMPORTANT] 

The focal and principal point extracted from the the results colmap execution

In [145]:

def projection(intrinsic_list, extrinsic, image_cloud, bounding_box):
    image_clouds = []
    depth = []
    # Get the intrinsic parameters of the camera
    focal_length = intrinsic_list
    principal_point = (960, 540)

    # Project the point cloud onto the image plane
    image_coordinates = image_cloud[:, :2] / image_cloud[:, 2:]
    image_coordinates[:, 0] = (image_coordinates[:, 0] * focal_length) + principal_point[0]
    image_coordinates[:, 1] = (image_coordinates[:, 1] * focal_length) + principal_point[1]
    # Extract the points inside the bounding box
    for bbox in bounding_box:
        x_min, y_min, x_max,  y_max = bbox
        inliers = np.where((image_coordinates[:, 0] > x_min) & 
                           (image_coordinates[:, 0] < x_max) & 
                           (image_coordinates[:, 1] > y_min) & 
                           (image_coordinates[:, 1] < y_max))
        image_cloud1 = image_coordinates[inliers]
        image_clouds.append(image_cloud1)
        
        dep = inverse_Homogenous_transformation(extrinsic, image_cloud[inliers])
    
        if(len(dep[:,2])>0):
            depth.append(round(np.min(dep[:,2])*ratio-1,1))
        else:
            depth.append(0)
       
    return image_clouds, depth

This is the main loop of this project which do the following

1- Align the point cloud with each image 

2- project the point cloud on the image

3- pick the points inside the bounding box (if exist look to challenges bellow)

4- the points inside each bounding box should represent the shape of the pothole so we can calculate the depth of the pothole by calculating the min of all the points which is logic to be the depth. For the size we considere it as a surface of a rectangle x*y which is in cm2

5- refer to the folder projectio to see the results of projection and also the depth and size value

In [146]:
for ind, image_name in enumerate(images):
    ext = extrisic_list[ind]
    ixt = intrinsic_list[ind]
    image_path = os.path.join(path, image_name)
    frame = cv2.imread(image_path)
    img_height, img_width, _ = frame.shape
    bounding_boxes = []
    
    if os.path.exists(os.path.join(path1,image_name[:-4]+'.txt')):
        with open(os.path.join(path1,image_name[:-4]+'.txt'), 'r') as file:
            for line in file:
                class_name, x, y, w, h = line.strip().split()
                bounding_boxes.append([float(x)*img_width, float(y)*img_height, float(w)*img_width, float(h)*img_height])
    if(len(bounding_boxes)>0):
        bounding_boxes = xywh2xyxy(np.array(bounding_boxes))
        image_clouds = Homogenous_transformation(ext, sc2_p)
        proj, depth = projection(ixt, ext, image_clouds, bounding_boxes)
        for j, bbox in enumerate(bounding_boxes):
            xmin, ymin, xmax, ymax = bbox
            draw_bounding_box_with_size_and_depth(frame, int(xmin), int(ymin), int(xmax), int(ymax), depth[j] , proj[j] )
        cv2.imwrite("projection/{}.jpg".format(ind+1), frame)
    
    

The severity now can be measured by the depth and size of the potholes

# Challenges

1 - knowing the ratio calculated above

2 - the quality of point cloud to improve

3 - some bounding box contain a few amount of points which do not give precision

4 - need a system to track each bounding box to optimze the calculation time because if a pothole is already calculated, there is no matter to recalculate it in next...

5 - Handling overlapping and inclusion between bounding box 

# Thanks